In [ ]:
import pandas as pd

# ItemIDとアイテム名の辞書
item_dict = {
    'I1': 'フランスワイン',
    'I2': 'イタリアワイン',
    'I3': 'スペインワイン',
    'I4': 'チリワイン',
    'I5': '国産ワイン'
}

# ワインの購買履歴のデータ
trans = [
    # TID: T1
    [1, 'I1'],
    [1, 'I2'],
    [1, 'I4'],
    [1, 'I5'],
    # TID: T2
    [2, 'I2'],
    [2, 'I3'],
    [2, 'I5'],
    # TID: T3
    [3, 'I1'],
    [3, 'I2'],
    [3, 'I4'],
    [3, 'I5'],
    # TID: T4
    [4, 'I1'],
    [4, 'I2'],
    [4, 'I3'],
    [4, 'I5'],
    # TID: T5
    [5, 'I1'],
    [5, 'I2'],
    [5, 'I3'],
    [5, 'I4'],
    [5, 'I5'],
    # TID: T6
    [6, 'I2'],
    [6, 'I3'],
    [6, 'I4']
]

# pandas DataFrame に変換、確認
df = pd.DataFrame(trans, columns=['TID', 'ItemID'])

# TIDを重複排除のため集合に変換
all_trans = set(df.TID)
# |D| : トランザクション数
all_trans_len = len(all_trans)

# 1つのitemについて、そのitemを含むTIDの集合を辞書で管理
count_dict = {}
# 全てのitemについて、そのitemを含むTIDの集合をtrans_setに追加
for item in df.ItemID.unique():
    count_dict[item] = set(df[df['ItemID'] == item].TID)

In [ ]:
from functools import reduce
from itertools import combinations

# サポートカウントの計算
def support_count(item_set):
    trans_set = []
    # item_setの各itemについて、そのitemを含むTIDの集合をtrans_setに追加
    for item in item_set:
        trans_set.append(count_dict[item])
    # trans_set の確認用
    # print(trans_set)
    # 全itemを含むTIDは、各itemを含むTIDの集合の積集合の要素数
    return len(reduce(lambda a, x: a & x, trans_set, all_trans))

# サポートの計算
def support(itemset):
    return support_count(itemset) / all_trans_len

# コンフィデンスの計算
def confidence(itemset_A, itemset_B):
    return support_count(itemset_A | itemset_B) / support_count(itemset_A)

# Generate_Candidates(L_{k-1}, k, min_sup_count)
def generate_candidates(L, k, min_sup_count):
    Ck = []
    for l1 in L:
        for l2 in L:
            # listのインデックスは 0 始まり
            if l1[0:k - 2] == l2[0:k - 2] and l1[k - 2] < l2[k - 2]:
                c = sorted(list(set(l1) | set(l2)))
                # print('k={}: {} | {} = {}'.format(k, l1, l2, c))
                for i in combinations(c, k - 1):
                    if support_count(i) < min_sup_count:
                        break
                else:   # 上のbreak文が実行されなかった場合
                    Ck.append(c)
    return Ck

# Apriori アルゴリズムの処理
def apriori():
    # 全てのアイテム
    all_items = sorted(list(set(df.ItemID)))

    # 頻出アイテムセットの集合を格納するdict型変数
    L = {}

    # L1 ← 1つのアイテムからなる頻出アイテムセットの集合
    L[1] = [[i] for i in all_items if support_count({i}) >= min_sup_count]
    # print('L1 = ', L[1])

    # k ← 2
    k = 2

    while L[k - 1] != []:
        Lk = []
        Ck = generate_candidates(L[k - 1], k, min_sup_count)
        for c in Ck:
            # print('{} {}'.format(c, support_count(c)))
            if support_count(c) >= min_sup_count:
                Lk.append(c)
        L[k] = Lk
        # print('L{} = {}'.format(k, L[k]))
        k = k + 1
    
    # 和集合
    L_union = []
    for ln in L.values():
        for l in ln:
            L_union.append(l)

    return L_union

# 相関ルールの生成
def create_assoc_rules(l, v=0):
    # 相関ルールの初期化
    assoc_rule = []
    # 全ての真部分集合 all_s のリストを初期化
    all_s = []
    # 全ての真部分集合 all_s の作成
    for n in range(1, len(l)):
        for i in combinations(l, n):
            all_s.append(list(i))
    # all_sの確認
    #print('all_s = {}'.format(all_s))

    # lのサポートカウント
    l_support_count = support_count(l)

    # 各真部分集合についてコンフィデンスを計算
    for s in all_s:
        # コンフィデンス(s ⇒ (l-s))
        s_support_count = support_count(s)
        cnfd = l_support_count / s_support_count
        # 確認用
        if (v > 0):
            print('{} {}/{}={}'.format(s, l_support_count, s_support_count, cnfd))
        # 最小コンフィデンス以上のルールを相関ルールに追加
        if cnfd >= min_confidence:
            # (l-s)
            l_s = [i for i in l if i not in s]
            sprt = l_support_count / all_trans_len
            lift = cnfd / (support_count(l_s) / all_trans_len)
            assoc_rule.append([s, l_s, sprt, cnfd, lift])

    return assoc_rule

# 全ての相関ルールの生成
def create_all_assoc_rules():
    # 相関ルールの初期化
    assoc_rules = []
    # Aprioriアルゴリズムで求めた全ての頻出アイテムセットから相関ルールを生成
    for l in apriori():
        rules = create_assoc_rules(l)
        if rules != []:
            for r in rules:
                assoc_rules.append(r)

    return pd.DataFrame(assoc_rules, columns=['L', 'R', 'support', 'confidence', 'lift'])

In [ ]:
# 最小サポート: 0.5
min_sup = 0.5
# 最小コンフィデンス
min_confidence = 0.7
# 最小サポートカウント
min_sup_count = min_sup * all_trans_len

# 相関ルールの確認
l = ['I2', 'I3', 'I5']
print('相関ルールの生成:', l)
create_assoc_rules(l, v=1)
#pd.DataFrame(create_assoc_rules(l, v=1), columns=['L', 'R', 'support', 'confidence', 'lift'])

In [ ]:
# 相関ルールの生成
print('全ての相関ルールの生成:')
df_rules = create_all_assoc_rules()
# 確認
df_rules

- support: 最小サポートが足切り規準となり、一定以上の重要性を持つルールだけが残されている
- confidence: 確信度が高いと、事象は一緒に起きやすい（商品は一緒に買われやすい）
- lift: 相関関係

In [ ]:
# lift値が1以上のルールについて、confidenceで降順にソートし、上位20を表示
df_rules.query('lift>=1').sort_values('confidence', ascending=False).head(20)

In [ ]:
# lift値が1以上, confidenceが1のルールについて、liftで降順にソートして表示
df_rules.query('lift>=1 and confidence==1').sort_values('lift', ascending=False)